<a href="https://colab.research.google.com/github/kcw0331/Environmental-Health-Big-Data-Analysis/blob/main/6%EC%A3%BC%EC%B0%A8/3-making_dataframes_from_api_requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_02

/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_02


# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [3]:
import datetime as dt
import pandas as pd
import requests  #requests를 이용해서 받아 올 거다.

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query' #이 주소로 가지고 온다.
payload = {
    'format': 'geojson', #format은 geojson으로 한다.
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

* starttime과 endtime을 사용해서 requests를 해준다.

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [4]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

* json형식으로 받은 다음에 형식을 살펴본다.

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [5]:
earthquake_json['metadata']

{'api': '1.12.2',
 'count': 10793,
 'generated': 1634290293000,
 'status': 200,
 'title': 'USGS Earthquakes',
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2021-09-14&endtime=2021-10-14'}

Each element in the JSON array `features` is a row of data for our dataframe.

In [6]:
type(earthquake_json['features'])

list

* features는 list형식으로 되어 있다.

Your data will be different depending on the date you run this.

In [7]:
earthquake_json['features'][0]

{'geometry': {'coordinates': [-155.477005004883,
   19.1853332519531,
   35.0900001525879],
  'type': 'Point'},
 'id': 'hv72753027',
 'properties': {'alert': None,
  'cdi': None,
  'code': '72753027',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=hv72753027&format=geojson',
  'dmin': None,
  'felt': None,
  'gap': 148,
  'ids': ',hv72753027,',
  'mag': 2.4000001,
  'magType': 'md',
  'mmi': None,
  'net': 'hv',
  'nst': 40,
  'place': '1 km S of P?hala, Hawaii',
  'rms': 0.129999995,
  'sig': 89,
  'sources': ',hv,',
  'status': 'automatic',
  'time': 1634169129840,
  'title': 'M 2.4 - 1 km S of P?hala, Hawaii',
  'tsunami': 0,
  'type': 'earthquake',
  'types': ',origin,phase-data,',
  'tz': None,
  'updated': 1634169320660,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/hv72753027'},
 'type': 'Feature'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

* features의 정보들을 dataframe에 잘 정리를 해준다.

In [8]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]

* earthquake_json['features']여기 안에서 quake로 받아서 quake의 properties를 가져와서 해주게 되면, earthquake_properties_data가 나오게 된다.

In [9]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,alert,status,tsunami,sig,net,code,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,2.40,"1 km S of P?hala, Hawaii",1634169129840,1634169320660,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,None,automatic,0,89,hv,72753027,",hv72753027,",",hv,",",origin,phase-data,",40.0,NaN,0.13,148.0,md,earthquake,"M 2.4 - 1 km S of P?hala, Hawaii"
1,0.96,"6 km WNW of Gorst, Washington",1634168634050,1634241117820,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,None,reviewed,0,14,uw,61789261,",uw61789261,",",uw,",",origin,phase-data,",5.0,0.003983,0.13,212.0,ml,explosion,"M 1.0 Explosion - 6 km WNW of Gorst, Washington"
2,1.30,"4km SW of Norco, CA",1634168483040,1634174427736,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,2.0,2.2,NaN,None,reviewed,0,26,ci,39833223,",ci39833223,",",ci,",",dyfi,nearby-cities,origin,phase-data,scitech-...",39.0,0.034120,0.13,52.0,ml,earthquake,"M 1.3 - 4km SW of Norco, CA"
3,1.03,"British Columbia, Canada",1634167963960,1634240721100,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,None,reviewed,0,16,uw,61789256,",uw61789256,",",uw,",",origin,phase-data,",7.0,0.071640,0.13,191.0,md,explosion,"M 1.0 Explosion - British Columbia, Canada"
4,1.78,"16 km SSE of Sunnyside, Utah",1634167723720,1634241329580,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,None,reviewed,0,49,uu,60463007,",uu60463007,",",uu,",",origin,phase-data,",6.0,0.021640,0.08,200.0,md,earthquake,"M 1.8 - 16 km SSE of Sunnyside, Utah"


* 이렇게 데이터 프레임으로 받아오고 그리고 마지막에는 to_csv를 사용해서 저장을 해준다.

### (Optional) Write Data to CSV

In [10]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>